In [1]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import( SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate )
from langchain_core.tools import tool

model_name = 'llama3.2:1b'
ollama_url = 'http://localhost:11434'
llm = ChatOllama(model=model_name, base_url=ollama_url)

In [2]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings('ignore')

load_dotenv('./../.env')

True

In [ ]:
### 개인 건강 전문가 (상담원)

In [8]:
# Tavily Search 함수 생성
from langchain_community.tools import TavilySearchResults

@tool
def tavily_search(query: str)->str:
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updatges etc.

    Args:
    query: The search query
    """

    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )

    response = search.invoke(query)
    return response

In [4]:
# 리트리버 생성 (PDF, FAISS)
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embed_model = 'nomic-embed-text'
db_path = './../09. Vector Stores and Retrievals/health_supplements'

embeddings = OllamaEmbeddings(model=embed_model, base_url=ollama_url)
vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})                  # 최근접 3개

In [5]:
# 리트리버 호출 함수 생성
@tool
def use_retriever(query:str)->str:
    """Search for information about Health Supplements.
    For any questions about Health and Gym Supplements, you must use this tool!,
    
    Args:
        query: The search query.
    """
    response = retriever.invoke(query)
    return response

In [6]:
use_retriever.invoke("what is the best supplement for muscle gain?")

[Document(id='71754c75-2a0a-4c9b-9c02-364db92e31d1', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:38:50+00:00', 'trapped': '', 'page': 0}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and high-\nintensity activities, which is the choice of consumers [4].\nCreatine monohydrate is another well-known supplement used to gain muscle mass\nand support performance an

In [7]:
### 에이전트 생성
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# '랭치엔 허브' 에서 Agent를 검색해서 프롬프트를 적절한 가져온다
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
tools = [tavily_search, use_retriever]
agent = create_tool_calling_agent(llm, tools, prompt)               # 어떤 도구를 호출할지 결정하는 에이전트 (답변생성이 아님)

In [11]:
# 어떤 도구를 호출할지 결정하는 에이전트 (답변생성이 아님)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [33]:
# question = "What is the best supplement for muscle gain?"
# question = "what's weather in Seoul?"
question = "What are the side effects of taking too much vitamin D?"
response = agent_executor.invoke({'input': question})           # 프롬프트 템플릿의 변수값이 input으로 명명되어있으므로



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'vitamin d toxicity side effects'}`


[{'url': 'https://www.healthline.com/nutrition/vitamin-d-side-effects', 'content': 'But too much vitamin D can also lead to health problems. Even when taking high dose vitamin D supplements, it’s unlikely that a healthy person’s blood vitamin D levels would come close to reaching excessive or toxic levels. The symptoms of vitamin D toxicity are primarily related to hypercalcemia, which means excessively high blood calcium levels. How much vitamin D is too much? How Much Vitamin D Is Too Much? Vitamin D is essential for health, but it can be hard to get enough of it from your diet. This is a detailed article about vitamin D and its health effects.'}, {'url': 'https://my.clevelandclinic.org/health/diseases/24750-vitamin-d-toxicity-hypervitaminosis-d', 'content': 'Vitamin D Toxicity: What It Is, Causes, Symptoms & Treatment What is vitamin D toxicity? Vitamin D toxicity

In [34]:
print(response['output'])

The side effects of taking too much vitamin D include:

* Hypercalcemia, which means excessively high blood calcium levels
* Nausea and vomiting
* Stomach pain
* Increased urination
* Excessive thirst

It's essential to note that excessive vitamin D intake is rare, and symptoms usually only occur when taking high doses of a vitamin D supplement or prescription medication. If you've been taking a vitamin D supplement and experience any of these symptoms, it's crucial to consult with a healthcare provider as soon as possible.
